### Importar Bibliotecas  

In [1]:
import requests
import pandas as pd
from datetime import datetime
import logging

### Função para obter os dados da API

In [2]:
def get_api_data(endpoint):
    url = f"https://fakestoreapi.com/{endpoint}"
    response = requests.get(url)
    return response.json()

### Configuração do logger para registro de erros

In [3]:
logging.basicConfig(filename='error.log', level=logging.ERROR)

### Obter todos os carrinhos de compras

In [4]:
carts = get_api_data("carts")

### Calcular o número total de carrinhos de compras

In [5]:
total_carts = len(carts)
print(f"Total de carrinhos: {total_carts}")

Total de carrinhos: 7


### Dicionário para armazenar informações dos usuários

In [6]:
user_data = {
    "id_usuario": [],
    "ultimo_carrinho": [],
    "principal_categoria": []
}

### Processamento dos dados, leitura e iterações.

In [7]:
#Iterar sobre os carrinhos de compras
for cart in carts:
    try:
        id_usuario = cart["userId"]
        cart_date = datetime.strptime(cart["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
        products = cart["products"]
        
        #Verificar se o usuário já está no dicionário
        if id_usuario not in user_data["id_usuario"]:
            #Adicionar uma nova entrada para cada usuário
            user_data["id_usuario"].append(id_usuario)
            user_data["ultimo_carrinho"].append(cart_date)
            
            # Se não houver produtos no carrinho, adicionar "Nenhuma" na coluna "principal_categoria"
            if not products:
                user_data["principal_categoria"].append("Nenhuma")
            else:
                # Para carrinhos não vazios, determinar a categoria principal
                categories = []
                for product in products:
                    product_id = product["productId"]
                    # Obter detalhes do produto usando o endpoint /products
                    product_details = get_api_data(f"products/{product_id}")
                    if "category" in product_details:
                        categories.append(product_details["category"])
                
                if categories:
                    most_common_category = max(set(categories), key=categories.count)
                    user_data["principal_categoria"].append(most_common_category)
                else:
                    user_data["principal_categoria"].append(None)
        else:
            #Verificar se o carrinho atual é mais recente
            idx = user_data["id_usuario"].index(id_usuario)
            if cart_date > user_data["ultimo_carrinho"][idx]:
                user_data["ultimo_carrinho"][idx] = cart_date
                #Atualizar as categorias se houver produtos no carrinho
                if products:
                    categories = []
                    for product in products:
                        product_id = product["productId"]
                        product_details = get_api_data(f"products/{product_id}")
                        if "category" in product_details:
                            categories.append(product_details["category"])
                    if categories:
                        most_common_category = max(set(categories), key=categories.count)
                        user_data["principal_categoria"][idx] = most_common_category
                    else:
                        user_data["principal_categoria"][idx] = None
    except Exception as e:
        logging.error(f"Erro ao processar o carrinho: {e}")

#Após a iteração, exibir o total de registros processados
total_records = len(carts)
print(f"Total de registros processados: {total_records}")


Total de registros processados: 7


### Criar DataFrame com os dados dos usuários

In [8]:
user_df = pd.DataFrame(user_data)
display(user_df)

,id_usuario,ultimo_carrinho,principal_categoria
0,1,2020-03-02,men's clothing
1,2,2020-03-01,men's clothing
2,3,2020-03-01,jewelery
3,4,2020-03-01,electronics
4,8,2020-03-01,women's clothing


### Salvar DataFrame em formato CSV

In [9]:
user_df.to_csv("user_data.csv", index=False)